In [3]:
import os
import numpy as np
import re
import pandas as pd
import scipy as sp
import random
import math

## Read and parse TSP data  

In [4]:
data = {}
for file in os.listdir('data'):
    with open(f'data/{file}') as f:
        name = f.readline().strip().split(' : ')[1]
        dim = f.readline().strip().split(' : ')[1]

        raw = f.readlines()
        raw = [list(map(float, re.split(r"\s+", line.strip()))) for line in raw]

        data[name] = raw

In [5]:
keys = data.keys()

## a) Create distance matrix

In [6]:
matrices = dict.fromkeys(keys, [])

for key in keys:
    dist = sp.spatial.distance_matrix(data[key], data[key])
    matrices[key] = dist

## Define Simple and Stochastic Hill Climbing algorithms

In [7]:
def generate_initial_tour(num_cities):
    tour = list(range(num_cities))
    random.shuffle(tour)
    return tour

def calculate_tour_length(tour, distances):
    length = 0
    for i in range(len(tour) - 1):
        length += distances[tour[i]][tour[i + 1]]
    length += distances[tour[-1]][tour[0]]
    return length

def get_neighbor(tour):
    i, j = random.sample(range(len(tour)), 2)
    tour[i], tour[j] = tour[j], tour[i]
    return tour

def simple_hill_climb(distances):
    num_cities = len(distances)
    current_tour = generate_initial_tour(num_cities)
    current_length = calculate_tour_length(current_tour, distances)

    for _ in range(10000):
        neighbor = get_neighbor(current_tour.copy())
        neighbor_length = calculate_tour_length(neighbor, distances)

        if neighbor_length < current_length:
            current_tour = neighbor
            current_length = neighbor_length

    return current_tour, current_length

def stochastic_hill_climb(distances):
    num_cities = len(distances)
    current_tour = generate_initial_tour(num_cities)
    current_length = calculate_tour_length(current_tour, distances)
    temperature = 1.0

    for _ in range(10000):
        neighbor = get_neighbor(current_tour.copy())
        neighbor_length = calculate_tour_length(neighbor, distances)

        delta_length = neighbor_length - current_length
        if delta_length < 0 or random.random() < pow(math.e, -(delta_length / temperature)):
            current_tour = neighbor
            current_length = neighbor_length

        temperature *= 0.95

    return current_tour, current_length

## b) Run the algorithms on generated matrices

In [13]:
final = {"Dataset": [],"Simple Hill Climb cost": [], "Stochastic Hill Climb cost": []}
for key, val in data.items():
    print(key)
    print()
    print("Simple Hill Climb")
    tour, shc_length = simple_hill_climb(matrices['rd100'])
    print(f"Tour: {tour}\nLength: {shc_length}")
    print()
    print("Stochastic Hill Climb")
    tour, sthc_length = stochastic_hill_climb(matrices['rd100'])
    print(f"Tour: {tour}\nLength: {sthc_length}")
    print("\n\n")

    final["Dataset"].append(key)
    final["Simple Hill Climb cost"].append(shc_length)
    final["Stochastic Hill Climb cost"].append(sthc_length)

a280

Simple Hill Climb
Tour: [89, 90, 63, 46, 52, 5, 78, 26, 53, 37, 69, 71, 18, 36, 92, 57, 75, 65, 83, 33, 6, 39, 41, 40, 21, 48, 20, 84, 81, 74, 2, 8, 25, 16, 49, 55, 27, 87, 88, 22, 1, 34, 28, 95, 54, 42, 24, 23, 29, 47, 68, 86, 61, 17, 0, 50, 64, 80, 94, 58, 76, 45, 72, 91, 9, 32, 3, 12, 35, 4, 60, 10, 56, 15, 51, 43, 44, 30, 97, 93, 98, 73, 77, 19, 31, 11, 13, 14, 7, 59, 70, 67, 99, 38, 79, 82, 66, 62, 85, 96]
Length: 16376.022993180477

Stochastic Hill Climb
Tour: [81, 74, 84, 31, 8, 25, 19, 49, 45, 75, 34, 28, 38, 54, 39, 44, 30, 65, 58, 57, 88, 1, 22, 43, 99, 51, 52, 5, 9, 32, 11, 13, 17, 70, 67, 82, 50, 90, 26, 36, 27, 92, 76, 94, 78, 2, 77, 73, 16, 53, 18, 55, 98, 93, 97, 15, 56, 80, 79, 48, 20, 3, 91, 72, 37, 69, 71, 12, 29, 47, 64, 89, 66, 96, 86, 61, 59, 68, 63, 35, 10, 60, 33, 4, 40, 21, 41, 6, 23, 24, 42, 95, 83, 87, 46, 85, 62, 0, 7, 14]
Length: 16785.012351874837



ch150

Simple Hill Climb
Tour: [51, 63, 12, 68, 67, 82, 50, 89, 64, 60, 33, 39, 54, 95, 34, 30, 57, 75

## c) Tabulate resultant tour length and cost data

In [14]:
df = pd.DataFrame(final)

In [15]:
df

,Dataset,Simple Hill Climb cost,Stochastic Hill Climb cost
0,a280,16376.022993,16785.012352
1,ch150,17483.462644,13752.057199
2,d198,16919.984758,17015.089545
3,eil101,16525.339742,17439.791870
4,rd100,14738.278250,15912.693682


In [16]:
shc_avg = df['Simple Hill Climb cost'].mean()
sto_avg = df['Stochastic Hill Climb cost'].mean()
shc_avg, sto_avg

(16408.6176775466, 16180.92892953832)

On average across all datasets, Stochastic Hill Climb performs better than Simple Hill Climb